### 语料，向量空间和模型的概念

一个语料库是数字文档的集合。 这个集合是gensim的输入，它将从中推断文档的结构或主题。  
从语料库中推断出的潜在结构（Latent Structure）可用于将主题分配给先前不存在于仅用于训练的语料库中的新文档。   
出于这个原因，我们也将此集合称为训练语料库（Training Corpus）  

In [1]:
import jieba


jieba.add_word('知识图谱') #防止“知识图谱”被切错词


raw_corpus = ['商业新知:知识图谱为内核,构建商业创新服务完整生态。',
'如何更好利用知识图谱技术做反欺诈? 360金融首席数据科学家沈赟开讲。',
'知识管理 | 基于知识图谱的国际知识管理领域可视化分析。',
'一文详解达观数据知识图谱技术与应用。',
'知识图谱技术落地金融行业的关键四步。',
'一文读懂知识图谱的商业应用进程及技术背景。',
'海云数据CPO王斌:打造大数据可视分析与AI应用的高科技企业。',
'智能产业|《人工智能标准化白皮书2018》带来创新创业新技术标准。',
'国家语委重大科研项目“中华经典诗词知识图谱构建技术研究”开题。',
'最全知识图谱介绍:关键技术、开放数据集、应用案例汇总。',
'中译语通Jove Mind知识图谱平台 引领企业智能化发展。',
'知识图谱:知识图谱赋能企业数字化转型，为企业升级转型注入新能量。']


raw_corpus = [' '.join(jieba.lcut(i)) for i in raw_corpus]  #对语料库中的文档进行分词，便于后续处理
raw_corpus

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\18438\AppData\Local\Temp\jieba.cache
Loading model cost 1.203 seconds.
Prefix dict has been built succesfully.


['商业 新知 : 知识图谱 为 内核 , 构建 商业 创新 服务 完整 生态 。',
 '如何 更好 利用 知识图谱 技术 做 反 欺诈 ?   360 金融 首席 数据 科学家 沈赟 开讲 。',
 '知识 管理   |   基于 知识图谱 的 国际 知识 管理 领域 可视化 分析 。',
 '一文 详解 达观 数据 知识图谱 技术 与 应用 。',
 '知识图谱 技术 落地 金融 行业 的 关键 四步 。',
 '一文 读懂 知识图谱 的 商业 应用 进程 及 技术 背景 。',
 '海云 数据 CPO 王斌 : 打造 大 数据 可视 分析 与 AI 应用 的 高科技 企业 。',
 '智能 产业 | 《 人工智能 标准化 白皮书 2018 》 带来 创新 创业 新 技术标准 。',
 '国家语委 重大 科研项目 “ 中华 经典 诗词 知识图谱 构建 技术 研究 ” 开题 。',
 '最全 知识图谱 介绍 : 关键技术 、 开放 数据 集 、 应用 案例 汇总 。',
 '中译 语通 Jove   Mind 知识图谱 平台   引领 企业 智能化 发展 。',
 '知识图谱 : 知识图谱 赋能 企业 数字化 转型 ， 为 企业 升级 转型 注入 新 能量 。']

收集语料库之后，通常会进行一系列的文本预处理。

In [13]:
stop_words = ['：', '。', '?', '|', '《', '》', ':', '“', '”', '，', '、']
# 移除常用词以及分词
stoplist = [i.strip() for i in stop_words]


#将文档中可能存在的西文字符小写化，按空格进行拆分，且去停用词
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]

#计算词频
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1


# 仅保留词频数高于1的词汇
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]

将语料库中的每个词汇与唯一的整数ID相关联。  
使用gensim.corpora.Dictionary这个类来完成。  

In [15]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(18 unique tokens: ['为', '创新', '商业', '构建', '知识图谱']...)


In [21]:
[x for x in dictionary.items()]

[(0, '为'),
 (1, '创新'),
 (2, '商业'),
 (3, '构建'),
 (4, '知识图谱'),
 (5, '技术'),
 (6, '数据'),
 (7, '金融'),
 (8, '分析'),
 (9, '的'),
 (10, '知识'),
 (11, '管理'),
 (12, '一文'),
 (13, '与'),
 (14, '应用'),
 (15, '企业'),
 (16, '新'),
 (17, '转型')]

#### 向量空间

为了推断语料库中的潜在结构（Latent Structure），我们需要一种可用于数学操作（比如，加减乘除等运算）的文档表示方法。  
一种方法是将每个文档表示为向量。有各种用于创建文档的向量表示的方法，其中一个简单的方法是词袋模型(Bag-of-Words Model)。

In [22]:
print(dictionary.token2id)

{'为': 0, '创新': 1, '商业': 2, '构建': 3, '知识图谱': 4, '技术': 5, '数据': 6, '金融': 7, '分析': 8, '的': 9, '知识': 10, '管理': 11, '一文': 12, '与': 13, '应用': 14, '企业': 15, '新': 16, '转型': 17}


doc2bow方法为该语句创建词袋表示，该方法返回词汇计数的稀疏表示：

In [23]:
new_doc = "知识图谱 为 企业 转型 助力"  #已分词，便于后续处理
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec  #第一个元素对应字典中的词汇ID 第二个元素对应该词汇的计数 #可以看到这里向量化后按照索引进行了排序 第一个不是“知识图谱”，而是“为”。

[(0, 1), (4, 1), (15, 1), (17, 1)]

#### 模型

In [28]:
from gensim import models
# 训练模型
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
tfidf = models.TfidfModel(bow_corpus)

tfidf[dictionary.doc2bow("知识图谱 为 企业 转型 助力".split())]

[(0, 0.5320703276607741),
 (4, 0.054141134526709565),
 (15, 0.4116657998035457),
 (17, 0.737903227562547)]

In [30]:
# 不同于TfidfVectorizer, TfidfMode的数据输入格式如下：需要先计算出每个词的索引和计数
bow_corpus

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1)],
 [(4, 1), (8, 1), (9, 1), (10, 2), (11, 2)],
 [(4, 1), (5, 1), (6, 1), (12, 1), (13, 1), (14, 1)],
 [(4, 1), (5, 1), (7, 1), (9, 1)],
 [(2, 1), (4, 1), (5, 1), (9, 1), (12, 1), (14, 1)],
 [(6, 2), (8, 1), (9, 1), (13, 1), (14, 1), (15, 1)],
 [(1, 1), (16, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(4, 1), (6, 1), (14, 1)],
 [(4, 1), (15, 1)],
 [(0, 1), (4, 2), (15, 2), (16, 1), (17, 2)]]

#### 文本向量的转换

In [31]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [32]:
# 训练模型
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
tfidf = models.TfidfModel(bow_corpus) # 通过打印出的日志可以看到，该TF-IDF转换模型在这12个经词袋表示的文档中抽取出了18个特征。

2022-03-25 15:02:21,809 : INFO : collecting document frequencies
2022-03-25 15:02:21,811 : INFO : PROGRESS: processing document #0
2022-03-25 15:02:21,812 : INFO : calculating IDF weights for 12 documents and 18 features (51 matrix non-zeros)


In [34]:
bow_corpus

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1)],
 [(4, 1), (8, 1), (9, 1), (10, 2), (11, 2)],
 [(4, 1), (5, 1), (6, 1), (12, 1), (13, 1), (14, 1)],
 [(4, 1), (5, 1), (7, 1), (9, 1)],
 [(2, 1), (4, 1), (5, 1), (9, 1), (12, 1), (14, 1)],
 [(6, 2), (8, 1), (9, 1), (13, 1), (14, 1), (15, 1)],
 [(1, 1), (16, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(4, 1), (6, 1), (14, 1)],
 [(4, 1), (15, 1)],
 [(0, 1), (4, 2), (15, 2), (16, 1), (17, 2)]]

In [38]:
doc_bow = [(0, 3), (1, 5)]
print(tfidf[doc_bow]) 

[(0, 0.5144957554275265), (1, 0.8574929257125441)]


#### 潜在语义索引

In [44]:
# step1 get processed_corpus
processed_corpus

[['商业', '知识图谱', '为', '构建', '商业', '创新'],
 ['知识图谱', '技术', '金融', '数据'],
 ['知识', '管理', '知识图谱', '的', '知识', '管理', '分析'],
 ['一文', '数据', '知识图谱', '技术', '与', '应用'],
 ['知识图谱', '技术', '金融', '的'],
 ['一文', '知识图谱', '的', '商业', '应用', '技术'],
 ['数据', '数据', '分析', '与', '应用', '的', '企业'],
 ['创新', '新'],
 ['知识图谱', '构建', '技术'],
 ['知识图谱', '数据', '应用'],
 ['知识图谱', '企业'],
 ['知识图谱', '知识图谱', '企业', '转型', '为', '企业', '转型', '新']]

In [43]:
# step2 processed_corpus > bow_corpus
bow_corpus

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1)],
 [(4, 1), (8, 1), (9, 1), (10, 2), (11, 2)],
 [(4, 1), (5, 1), (6, 1), (12, 1), (13, 1), (14, 1)],
 [(4, 1), (5, 1), (7, 1), (9, 1)],
 [(2, 1), (4, 1), (5, 1), (9, 1), (12, 1), (14, 1)],
 [(6, 2), (8, 1), (9, 1), (13, 1), (14, 1), (15, 1)],
 [(1, 1), (16, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(4, 1), (6, 1), (14, 1)],
 [(4, 1), (15, 1)],
 [(0, 1), (4, 2), (15, 2), (16, 1), (17, 2)]]

In [40]:
# step3 bow_corpus > corpus_tfidf
corpus_tfidf = tfidf[bow_corpus]

我们通过潜在语义索引（Latent Semantic Indexing）将前面转换得到的TF-IDF语料库转换到潜在的3-D空间（3-D因为笔者在这里设置了num_topics = 3）。   

In [41]:
# step4 corpus_tfidf > Lsi
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=3) # 初始化 LSI 转换
corpus_lsi = lsi[corpus_tfidf] 

2022-03-25 15:07:57,180 : INFO : using serial LSI version on this node
2022-03-25 15:07:57,181 : INFO : updating model with new documents
2022-03-25 15:07:57,184 : INFO : preparing a new chunk of documents
2022-03-25 15:07:57,188 : INFO : using 100 extra samples and 2 power iterations
2022-03-25 15:07:57,190 : INFO : 1st phase: constructing (18, 103) action matrix
2022-03-25 15:07:57,196 : INFO : orthonormalizing (18, 103) action matrix
2022-03-25 15:07:57,211 : INFO : 2nd phase: running dense svd on (18, 12) matrix
2022-03-25 15:07:57,217 : INFO : computing the final decomposition
2022-03-25 15:07:57,219 : INFO : keeping 3 factors (discarding 50.060% of energy spectrum)
2022-03-25 15:07:57,221 : INFO : processed documents up to #12
2022-03-25 15:07:57,226 : INFO : topic #0(1.667): 0.522*"数据" + 0.415*"应用" + 0.339*"技术" + 0.327*"金融" + 0.283*"一文" + 0.275*"与" + 0.244*"的" + 0.186*"商业" + 0.184*"企业" + 0.127*"分析"
2022-03-25 15:07:57,227 : INFO : topic #1(1.276): 0.655*"企业" + -0.459*"金融" + -0.3

现在你可能想知道：这3个潜在的维度代表什么？ 让我们用models.LsiModel.show_topics()一探究竟：

In [42]:
lsi.show_topics()

[(0,
  '0.522*"数据" + 0.415*"应用" + 0.339*"技术" + 0.327*"金融" + 0.283*"一文" + 0.275*"与" + 0.244*"的" + 0.186*"商业" + 0.184*"企业" + 0.127*"分析"'),
 (1,
  '0.655*"企业" + -0.459*"金融" + -0.317*"技术" + 0.294*"转型" + -0.190*"构建" + 0.173*"新" + 0.151*"与" + 0.145*"应用" + 0.127*"分析" + 0.126*"数据"'),
 (2,
  '0.500*"商业" + 0.456*"构建" + 0.414*"创新" + -0.305*"数据" + 0.289*"新" + 0.260*"为" + 0.187*"转型" + -0.148*"应用" + 0.141*"企业" + -0.131*"与"')]

由于LSI、LDA等主题模型在本质上属于软聚类（Soft Clustering），也就是说，每个主题上的概率就是文章对于这个主题的隶属度，同一个文档可能夹杂着多个主题，只是对应的各个主题的概率不同罢了。   
上述文档对应的主题应该是混合的，每个主题的概率大小不尽相同，在实际应用中，我们一般找出其中概率最大的一个主题。

In [45]:
for doc in corpus_lsi: # 显示每个文档归属于三个主题的概率
    print(doc)

[(0, 0.2205980967355469), (1, -0.0915199211338029), (2, 0.8064302977931628)]
[(0, 0.6470631618916501), (1, -0.4193056582509369), (2, -0.1422387347128287)]
[(0, 0.10495793818518592), (1, 0.033973174654206906), (2, -0.02988420923937445)]
[(0, 0.7558863710004506), (1, 0.08278059466237589), (2, -0.165519159603095)]
[(0, 0.5131637299454848), (1, -0.5333413310451962), (2, 0.01940115646669755)]
[(0, 0.6060694248904178), (1, -0.1417807816122505), (2, 0.3176976421755372)]
[(0, 0.7208233984239172), (1, 0.43579976296097106), (2, -0.24672666696702383)]
[(0, 0.049205047521911065), (1, 0.1552595650274925), (2, 0.49733526951599605)]
[(0, 0.26054052006849115), (1, -0.3064643335040266), (2, 0.4681367948232273)]
[(0, 0.6716514876891511), (1, 0.19273932762594662), (2, -0.3127253597472357)]
[(0, 0.19745672369019995), (1, 0.652210121288744), (2, 0.14590420350541577)]
[(0, 0.14129975671133854), (1, 0.600309157305573), (2, 0.3717412874506636)]


#### 其他文本向量转换方法

随机映射（Random Projections）

RP算法旨在减少向量空间的维度。 这是一种非常有效的（对内存和CPU友好）方法，通过加入一点随机性来近似表示文档之间的TF-IDF距离。 比较推荐的目标维度最好是几百/几千，具体数值取决于语料库的大小。

In [46]:
model = models.RpModel(corpus_tfidf, num_topics=3)

2022-03-25 15:14:53,363 : INFO : no word id mapping provided; initializing from corpus, assuming identity
2022-03-25 15:14:53,366 : INFO : constructing (3, 18) random matrix


In [47]:
# 文档的主题数
model.num_topics

3

In [48]:
# 主题模型中参与训练的词汇数有多少？
model.num_terms 

18

In [53]:
[x for x in model[corpus_tfidf]]

[[(0, 0.6319815516471863), (1, -0.6319815516471863), (2, -0.6319815516471863)],
 [(0, 0.46322357654571533), (1, 0.09217122197151184), (2, 0.9979811906814575)],
 [(0, -0.06887989491224289),
  (1, -0.7131474018096924),
  (2, -0.21306325495243073)],
 [(0, 0.21194177865982056), (1, 0.6054527759552002), (2, 1.2720093727111816)],
 [(0, 0.4632236063480377), (1, -0.4632236063480377), (2, 0.44258639216423035)],
 [(0, 0.21194176375865936), (1, 0.196755513548851), (2, 0.196755513548851)],
 [(0, 0.845433235168457), (1, 0.11859847605228424), (2, 0.5240893363952637)],
 [(0, -0.8164966106414795), (1, 0.8164966106414795)],
 [(0, 0.21161195635795593),
  (1, -0.21161195635795593),
  (2, 0.8215587139129639)],
 [(0, 0.7436424493789673), (1, 0.8782217502593994), (2, 0.8782217502593994)],
 [(0, 0.49713781476020813),
  (1, -0.49713781476020813),
  (2, 0.6477042436599731)],
 [(0, 0.6825988292694092),
  (1, -0.6825988292694092),
  (2, -0.16955256462097168)]]

隐狄利克雷分配模型（Latent Dirichlet Allocation, LDA）

In [57]:
model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=3)

2022-03-25 15:18:14,394 : INFO : using symmetric alpha at 0.3333333333333333
2022-03-25 15:18:14,397 : INFO : using symmetric eta at 0.3333333333333333
2022-03-25 15:18:14,398 : INFO : using serial LDA version on this node
2022-03-25 15:18:14,403 : INFO : running online (single-pass) LDA training, 3 topics, 1 passes over the supplied corpus of 12 documents, updating model once every 12 documents, evaluating perplexity every 12 documents, iterating 50x with a convergence threshold of 0.001000
2022-03-25 15:18:14,407 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2022-03-25 15:18:14,426 : INFO : -5.027 per-word bound, 32.6 perplexity estimate based on a held-out corpus of 12 documents with 21 words
2022-03-25 15:18:14,427 : INFO : PROGRESS: pass 0, at document #12/12
2022-03-25 15:18:14,444 : INFO : topic #0 (0.333): 0.136*"金融" + 0.115*"技术" + 0.096*"构建" + 0.081*"创新" + 0.075*"新" + 0.068*"数据" + 0.061*"的" 

In [58]:
# 展示主题模型中的各个主体及其主题词分布情况：
model.show_topics()

[(0,
  '0.136*"金融" + 0.115*"技术" + 0.096*"构建" + 0.081*"创新" + 0.075*"新" + 0.068*"数据" + 0.061*"的" + 0.048*"知识图谱" + 0.040*"企业" + 0.035*"应用"'),
 (1,
  '0.101*"转型" + 0.096*"商业" + 0.084*"为" + 0.081*"企业" + 0.065*"创新" + 0.061*"新" + 0.061*"构建" + 0.047*"与" + 0.047*"数据" + 0.044*"应用"'),
 (2,
  '0.109*"应用" + 0.107*"数据" + 0.088*"企业" + 0.082*"一文" + 0.070*"与" + 0.068*"的" + 0.060*"分析" + 0.059*"管理" + 0.059*"知识" + 0.054*"商业"')]

层次狄利克雷过程（Hierarchical Dirichlet Process, HDP）

HDP是一种非参数贝叶斯方法（跟其他的主题模型不同，它不需要事先确定主题的数量）

In [59]:
model = models.HdpModel(corpus_tfidf, id2word=dictionary)

2022-03-25 15:19:13,499 : WARNING : likelihood is decreasing!
2022-03-25 15:19:13,501 : WARNING : likelihood is decreasing!
2022-03-25 15:19:13,510 : INFO : (0, '0.164*知识 + 0.149*管理 + 0.096*应用 + 0.096*技术 + 0.092*构建 + 0.079*转型 + 0.064*数据 + 0.051*商业 + 0.045*知识图谱 + 0.040*创新')
2022-03-25 15:19:13,512 : INFO : (1, '0.247*企业 + 0.161*商业 + 0.078*知识 + 0.070*技术 + 0.070*金融 + 0.061*分析 + 0.059*与 + 0.054*应用 + 0.036*数据 + 0.033*一文')
2022-03-25 15:19:13,513 : INFO : (2, '0.181*创新 + 0.166*金融 + 0.121*分析 + 0.093*企业 + 0.081*一文 + 0.076*商业 + 0.048*的 + 0.046*数据 + 0.036*与 + 0.035*转型')
2022-03-25 15:19:13,513 : INFO : (3, '0.175*的 + 0.132*管理 + 0.106*与 + 0.099*构建 + 0.079*创新 + 0.074*一文 + 0.050*分析 + 0.047*数据 + 0.046*为 + 0.040*知识')
2022-03-25 15:19:13,514 : INFO : (4, '0.273*技术 + 0.138*与 + 0.110*创新 + 0.105*一文 + 0.069*为 + 0.065*知识 + 0.035*企业 + 0.034*商业 + 0.033*分析 + 0.033*应用')
2022-03-25 15:19:13,517 : INFO : (5, '0.201*分析 + 0.149*商业 + 0.124*一文 + 0.108*管理 + 0.098*构建 + 0.044*应用 + 0.040*数据 + 0.036*技术 + 0.035*知识 + 0.029

#### 主题模型的主题数确定与可视化

In [74]:
from gensim.corpora import Dictionary
from gensim.models import ldamodel
from gensim.models import CoherenceModel, LdaModel
from gensim import models
import numpy
%matplotlib inline

In [75]:
texts = [
            ['苹果','叶子','椭圆形','树上'],
            ['植物','叶子','绿色','落叶乔木'],
            ['水果','苹果','红彤彤','味道'],
            ['苹果','落叶乔木','树上','水果'],
            ['植物','营养','水果','维生素'],
            ['营养','维生素','苹果','成分'],
            ['互联网','电脑','智能手机','高科技'],
            ['苹果','公司','互联网','品质'],
            ['乔布斯','苹果','硅谷'],
            ['电脑','智能手机','苹果','乔布斯'], 
            ['苹果','电脑','品质','生意'],
            ['电脑','品质','乔布斯'],
            ['苹果','公司','生意','硅谷']

            ]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2022-03-25 15:31:20,784 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-25 15:31:20,787 : INFO : built Dictionary(22 unique tokens: ['叶子', '树上', '椭圆形', '苹果', '植物']...) from 13 documents (total 50 corpus positions)


接下来，笔者将训练两个主题模型，差异在于主题数的不同，按照笔者构建的语料库构成来看，主题数应该是2，假如是其他的主题数，模型的效果应该不好。  
下面，基于假设，“好”的主题模型的主题数为2，“坏”的主题模型的主题数为6。

In [76]:
numpy.random.seed(1) # 设置随即种子数，以便相同的设置能跑出相同的结果，可复现
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary,
     iterations=50, num_topics=2)
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, 
     iterations=50, num_topics=6)

2022-03-25 15:31:52,768 : INFO : using symmetric alpha at 0.5
2022-03-25 15:31:52,770 : INFO : using symmetric eta at 0.5
2022-03-25 15:31:52,772 : INFO : using serial LDA version on this node
2022-03-25 15:31:52,775 : INFO : running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 13 documents, updating model once every 13 documents, evaluating perplexity every 13 documents, iterating 50x with a convergence threshold of 0.001000
2022-03-25 15:31:52,777 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2022-03-25 15:31:52,805 : INFO : -3.928 per-word bound, 15.2 perplexity estimate based on a held-out corpus of 13 documents with 50 words
2022-03-25 15:31:52,806 : INFO : PROGRESS: pass 0, at document #13/13
2022-03-25 15:31:52,823 : INFO : topic #0 (0.500): 0.134*"苹果" + 0.067*"水果" + 0.064*"公司" + 0.064*"落叶乔木" + 0.063*"叶子" + 0.062*"植物" + 0.061*"树上" + 0.054*"生意" + 0.052*"品质" 

通过指标确定合理的主题数

1 使用U_Mass Coherence


In [77]:
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, 
       dictionary=dictionary, coherence='u_mass')
badcm = CoherenceModel(model=badLdaModel, corpus=corpus,
       dictionary=dictionary, coherence='u_mass')

print(goodcm.get_coherence())
print(badcm.get_coherence()) #虽然数值差异不大，但仍能看出“好”的主题模型的U_Mass Coherence要大于坏的模型的数值。

-18.78459503442916
-18.83035774373808


2 使用 C_V coherence

In [78]:
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, 
    dictionary=dictionary,  coherence='c_v')
badcm = CoherenceModel(model=badLdaModel, texts=texts,
    dictionary=dictionary,  coherence='c_v')

print(goodcm.get_coherence())
print(badcm.get_coherence()) # 跟上述结果一样，虽然数值差异不大，但“好”的主题模型的C_V coherence值仍要大于“坏”的主题模型的C_V coherence值

2022-03-25 15:33:04,507 : INFO : using ParallelWordOccurrenceAccumulator(processes=11, batch_size=64) to estimate probabilities from sliding windows
2022-03-25 15:33:08,222 : INFO : 11 accumulators retrieved from output queue
2022-03-25 15:33:08,247 : INFO : accumulated word occurrence stats for 13 virtual documents
2022-03-25 15:33:08,344 : INFO : using ParallelWordOccurrenceAccumulator(processes=11, batch_size=64) to estimate probabilities from sliding windows


0.5880602397643417


2022-03-25 15:33:11,767 : INFO : 11 accumulators retrieved from output queue
2022-03-25 15:33:11,785 : INFO : accumulated word occurrence stats for 13 virtual documents


0.5868870960312388


预测词汇的主题归属

In [80]:
model = ldamodel.LdaModel(corpus, id2word=dictionary,
           iterations=500,num_topics=2,alpha='auto')

print(model.show_topics(num_words=5))

#根据主题模型运行出来的结果，序号为0的暂定为“水果”，序号为1的暂定为“公司”，用来测试几个词汇的主题归属情况
topic_list = ['水果','公司']

[(topic_list[i[0]],i[1]) for i in model.get_term_topics('树上')] # “树上”这个词汇更加靠近“水果”这个主题。

2022-03-25 15:35:54,000 : INFO : using autotuned alpha, starting with [0.5, 0.5]
2022-03-25 15:35:54,002 : INFO : using symmetric eta at 0.5
2022-03-25 15:35:54,004 : INFO : using serial LDA version on this node
2022-03-25 15:35:54,005 : INFO : running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 13 documents, updating model once every 13 documents, evaluating perplexity every 13 documents, iterating 500x with a convergence threshold of 0.001000
2022-03-25 15:35:54,006 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2022-03-25 15:35:54,031 : INFO : -3.922 per-word bound, 15.2 perplexity estimate based on a held-out corpus of 13 documents with 50 words
2022-03-25 15:35:54,032 : INFO : PROGRESS: pass 0, at document #13/13
2022-03-25 15:35:54,057 : INFO : optimized alpha [0.6690984, 0.7567388]
2022-03-25 15:35:54,060 : INFO : topic #0 (0.669): 0.143*"苹果" + 0.077*"水果" +

[(0, '0.143*"苹果" + 0.077*"水果" + 0.070*"树上" + 0.058*"电脑" + 0.048*"叶子"'), (1, '0.135*"苹果" + 0.078*"电脑" + 0.075*"乔布斯" + 0.074*"品质" + 0.055*"植物"')]


[('水果', 0.05612464)]

预测文档的主题归属

In [82]:
bow_fruit = ['苹果','树上','落叶乔木','苹果']
bow_company = ['苹果','电脑','乔布斯']

bow = model.id2word.doc2bow(bow_fruit)     # 现将文档转换为词袋表示
doc_topics, word_topics, phi_values = model.get_document_topics(bow, per_word_topics=True)

word_topics

[(1, [0, 1]), (3, [0, 1]), (6, [0, 1])]

上面的结果应该这样理解：1、3、6对应文档bow_fruit中的3个词汇：'苹果'、'树上'、'落叶乔木'，它们的主题更加倾向于是“水果”，因为每个词汇的主题序号中，0都排在靠前的位置，也就是“水果”这个主题更为明显。  
get_document_topics这个方法产生了3个值doc_topics、 word_topics和 phi_values。  
对于phi_values而言，它包含特定词汇在各主题上的phi值，且按特征长度缩放。 Phi本质上是文档中某个词汇从属于某个主题的概率值。

In [84]:
phi_values

[(1, [(0, 0.9572513), (1, 0.042745788)]),
 (3, [(0, 1.5801868), (1, 0.4198111)]),
 (6, [(0, 0.789432), (1, 0.21056363)])]

通过get_document_topics来获取语料库中所有文档的“doc_topics”，“word_topics”和“phi_values”：



In [85]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(0, 1), (4, 1), (5, 1), (6, 1)],
 [(3, 1), (7, 1), (8, 1), (9, 1)],
 [(1, 1), (3, 1), (6, 1), (8, 1)],
 [(4, 1), (8, 1), (10, 1), (11, 1)],
 [(3, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (14, 1), (15, 1), (16, 1)],
 [(3, 1), (13, 1), (17, 1), (18, 1)],
 [(3, 1), (19, 1), (20, 1)],
 [(3, 1), (14, 1), (15, 1), (19, 1)],
 [(3, 1), (15, 1), (18, 1), (21, 1)],
 [(15, 1), (18, 1), (19, 1)],
 [(3, 1), (17, 1), (20, 1), (21, 1)]]

In [87]:
all_topics = model.get_document_topics(corpus, per_word_topics=True)

cnt = 0
for doc_topics, word_topics, phi_values in all_topics:
    print('新文档:{} \n'.format(cnt),texts[cnt])
    doc_topics = [(topic_list[i[0]],i[1]) for i in doc_topics]
    word_topics = [(dictionary.id2token [i[0]],i[1]) for i in word_topics]
    phi_values = [(dictionary.id2token [i[0]],i[1]) for i in phi_values ]
    print('文档主题:', doc_topics)
    print('词汇主题:', word_topics)
    print('Phi值:', phi_values)
    print(" ")
    print('-------------- \n')
    cnt+=1

新文档:0 
 ['苹果', '叶子', '椭圆形', '树上']
文档主题: [('水果', 0.802878), ('公司', 0.19712195)]
词汇主题: [('叶子', [0, 1]), ('树上', [0, 1]), ('椭圆形', [0, 1]), ('苹果', [0, 1])]
Phi值: [('叶子', [(0, 0.88831884), (1, 0.11167704)]), ('树上', [(0, 0.97451615), (1, 0.025481217)]), ('椭圆形', [(0, 0.9592283), (1, 0.040766396)]), ('苹果', [(0, 0.8653774), (1, 0.1346216)])]
 
-------------- 

新文档:1 
 ['植物', '叶子', '绿色', '落叶乔木']
文档主题: [('水果', 0.1982791), ('公司', 0.80172086)]
词汇主题: [('叶子', [1, 0]), ('植物', [1, 0]), ('绿色', [1, 0]), ('落叶乔木', [1, 0])]
Phi值: [('叶子', [(0, 0.17409904), (1, 0.825896)]), ('植物', [(0, 0.0402802), (1, 0.9597165)]), ('绿色', [(0, 0.046808325), (1, 0.9531854)]), ('落叶乔木', [(0, 0.14506549), (1, 0.85493)])]
 
-------------- 

新文档:2 
 ['水果', '苹果', '红彤彤', '味道']
文档主题: [('水果', 0.81959784), ('公司', 0.18040214)]
词汇主题: [('苹果', [0, 1]), ('味道', [0, 1]), ('水果', [0, 1]), ('红彤彤', [0, 1])]
Phi值: [('苹果', [(0, 0.8837181), (1, 0.116280995)]), ('味道', [(0, 0.9744483), (1, 0.025546927)]), ('水果', [(0, 0.9437744), (1, 0.056223415)]), ('红彤